In [1]:
import duckdb
import pandas as pd

%load_ext sql
# Connect to an in-memory database
conn = duckdb.connect()
%sql conn --alias duckdb

In [2]:
# Initialize TPCH extension (it should come with the duckdb installation)
%sql CREATE OR REPLACE SCHEMA SF_1;
%sql USE SF_1;
%sql CALL dbgen(sf =1)

Running query in 'duckdb'

Running query in 'duckdb'

Running query in 'duckdb'

Success


In [3]:
%sql show tables;

Running query in 'duckdb'

name
customer
lineitem
nation
orders
part
partsupp
region
supplier


In [4]:
#%sql FROM tpch_queries();
res = conn.sql('FROM tpch_queries()').df()
query = res.iloc[18]['query']
print(query)

SELECT
    sum(l_extendedprice * (1 - l_discount)) AS revenue
FROM
    lineitem,
    part
WHERE (p_partkey = l_partkey
    AND p_brand = 'Brand#12'
    AND p_container IN ('SM CASE', 'SM BOX', 'SM PACK', 'SM PKG')
    AND l_quantity >= 1
    AND l_quantity <= 1 + 10
    AND p_size BETWEEN 1 AND 5
    AND l_shipmode IN ('AIR', 'AIR REG')
    AND l_shipinstruct = 'DELIVER IN PERSON')
    OR (p_partkey = l_partkey
        AND p_brand = 'Brand#23'
        AND p_container IN ('MED BAG', 'MED BOX', 'MED PKG', 'MED PACK')
        AND l_quantity >= 10
        AND l_quantity <= 10 + 10
        AND p_size BETWEEN 1 AND 10
        AND l_shipmode IN ('AIR', 'AIR REG')
        AND l_shipinstruct = 'DELIVER IN PERSON')
    OR (p_partkey = l_partkey
        AND p_brand = 'Brand#34'
        AND p_container IN ('LG CASE', 'LG BOX', 'LG PACK', 'LG PKG')
        AND l_quantity >= 20
        AND l_quantity <= 20 + 10
        AND p_size BETWEEN 1 AND 15
        AND l_shipmode IN ('AIR', 'AIR REG')
        

In [9]:
%sql DESCRIBE lineitem;

column_name,column_type,null,key,default,extra
l_orderkey,BIGINT,NO,None,None,None
l_partkey,BIGINT,NO,None,None,None
l_suppkey,BIGINT,NO,None,None,None
l_linenumber,BIGINT,NO,None,None,None
l_quantity,"DECIMAL(15,2)",NO,None,None,None
l_extendedprice,"DECIMAL(15,2)",NO,None,None,None
l_discount,"DECIMAL(15,2)",NO,None,None,None
l_tax,"DECIMAL(15,2)",NO,None,None,None
l_returnflag,VARCHAR,NO,None,None,None
l_linestatus,VARCHAR,NO,None,None,None


In [11]:
%sql DESCRIBE orders;

column_name,column_type,null,key,default,extra
o_orderkey,BIGINT,NO,None,None,None
o_custkey,BIGINT,NO,None,None,None
o_orderstatus,VARCHAR,NO,None,None,None
o_totalprice,"DECIMAL(15,2)",NO,None,None,None
o_orderdate,DATE,NO,None,None,None
o_orderpriority,VARCHAR,NO,None,None,None
o_clerk,VARCHAR,NO,None,None,None
o_shippriority,INTEGER,NO,None,None,None
o_comment,VARCHAR,NO,None,None,None


In [5]:
%sql DESCRIBE part;

Running query in 'duckdb'

column_name,column_type,null,key,default,extra
p_partkey,BIGINT,NO,None,None,None
p_name,VARCHAR,NO,None,None,None
p_mfgr,VARCHAR,NO,None,None,None
p_brand,VARCHAR,NO,None,None,None
p_type,VARCHAR,NO,None,None,None
p_size,INTEGER,NO,None,None,None
p_container,VARCHAR,NO,None,None,None
p_retailprice,"DECIMAL(15,2)",NO,None,None,None
p_comment,VARCHAR,NO,None,None,None


In [15]:
# Configure cell to output plain text
%config SqlMagic.displaycon = True
%config SqlMagic.displaylimit = 100

In [4]:
%sql SELECT DATE '1992-03-22' as date;

Running query in 'duckdb'

date
1992-03-22


In [17]:
%%sql explain SELECT
o_orderpriority,
count(*) AS order_count
FROM
orders
WHERE
o_orderdate >= CAST('1993-07-01' AS date)
AND o_orderdate < CAST('1993-10-01' AS date)
AND EXISTS (
SELECT
*
FROM
lineitem
WHERE
l_orderkey = o_orderkey
AND l_commitdate < l_receiptdate)
GROUP BY
o_orderpriority
ORDER BY
o_orderpriority;

Running query in 'duckdb'

+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------+
|  explain_key  |                                                                   explain_value                                                                   |
+---------------+---------------------------------------------------------------------------------------------------------------------------------------------------+
| physical_plan |                ┌───────────────────────────┐                                                                                                      |
|               |                │         PROJECTION        │                                                                                                      |
|               |                │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                                      |
|               |                │__internal_decompress_strin│                                                                                                      |
|               |                │           g(#0)           │                                                                                                      |
|               |                │             #1            │                                                                                                      |
|               | └─────────────┬─────────────┘                                                                                                                     |
|               |                ┌─────────────┴─────────────┐                                                                                                      |
|               |                │          ORDER_BY         │                                                                                                      |
|               |                │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                                      |
|               |                │          ORDERS:          │                                                                                                      |
|               |                │ orders.o_orderpriority ASC│                                                                                                      |
|               | └─────────────┬─────────────┘                                                                                                                     |
|               |                ┌─────────────┴─────────────┐                                                                                                      |
|               |                │         PROJECTION        │                                                                                                      |
|               |                │   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │                                                                                                      |
|               |                │__internal_compress_string_│                                                                                                      |
|               |                │        hugeint(#0)        │                                                                                                      |
|               |                │             #1            │                                                                                                      |
|               | └─────────────┬─────────────┘                                                                                                                     |
|               |                ┌─────────────┴─────────────┐                                                                                                      |
|               